In [6]:
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from Levenshtein import distance as levenshtein_distance
import h5py
import matplotlib.pyplot as plt

In [ ]:
name = "1a1zA00"
temp = "320"
replica = "0"

with h5py.File('../tmp/data/tokenized/mdcath/mdcath_tokenized_foldseek.h5', 'r') as file:
    tokenized_mdcath = {
        f"{protein}_{trajectory}": [x.decode('utf-8') for x in file['foldseek'][protein][trajectory]]
        for protein in list(file['foldseek'].keys())[:25]
        for trajectory in list(file['foldseek'][protein].keys())
    }
print(*tokenized_mdcath[f'{name}_{temp}_{replica}'][164:167], sep='\n')


# First Frame

In [ ]:

u = mda.Universe(f"../tmp/output/trajectories/{name}.pdb", 
                 f"../tmp/output/trajectories/{name}_{temp}_{replica}.xtc")

n_frames = len(u.trajectory)
dt = u.trajectory.dt
time = np.arange(n_frames) * dt

rmsd_values = np.zeros(n_frames)

u.trajectory[0]
first_frame_coords = u.select_atoms('backbone').positions.copy()

ref = u.copy()
ref.trajectory[0]

aligner = align.AlignTraj(u, ref, select='backbone', in_memory=True)
aligner.run()

for i, ts in enumerate(u.trajectory):
    current_pos = u.select_atoms('backbone').positions
    rmsd = rms.rmsd(current_pos, first_frame_coords)
    rmsd_values[i] = rmsd

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=rmsd_values,
                        mode='lines',
                        name='RMSD'))

fig.update_layout(
    title=f'{name} Backbone RMSD vs First Frame',
    xaxis_title='Time (ps)',
    yaxis_title='RMSD (Å)',
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [ ]:
distances = []
dist_name = "Levenshtein"
for i in range(0, len(tokenized_mdcath[f'{name}_{temp}_{replica}'])):
    distance = levenshtein_distance(tokenized_mdcath[f'{name}_{temp}_{replica}'][0], tokenized_mdcath[f'{name}_{temp}_{replica}'][i])
    distances.append(distance)

fig = go.Figure()
fig.add_trace(go.Scatter(x=time[1:], y=distances,
                        mode='lines',
                        name=dist_name))

fig.update_layout(
    title=f'{dist_name} Distance vs First Frame',
    xaxis_title='Time (ps)',
    yaxis_title=f'{dist_name} Distance',
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [ ]:
fig = go.Figure()

start_idx = 1
fig.add_trace(go.Scatter(x=time[start_idx:], y=rmsd_values[start_idx:],
                        mode='lines',
                        name='RMSD',
                        yaxis='y2'))
fig.add_trace(go.Scatter(x=time[start_idx:], y=distances[start_idx:],
                        mode='lines',
                        name=dist_name))

fig.update_layout(
    yaxis2=dict(
        overlaying='y',
        side='right',
        title='RMSD (Å)'
    )
)

fig.update_layout(
    yaxis=dict(
        title=f'{dist_name} Distance'
    )
)

fig.update_layout(
    title=f'{name} RMSD and Token Levenshtein Distance vs First Frame',
    xaxis_title='Time (ps)',
    showlegend=True,
    template='plotly_white'
)

# Consecutive Frames

In [ ]:
u = mda.Universe(f"../tmp/output/trajectories/{name}.pdb", 
                 f"../tmp/output/trajectories/{name}_{temp}_{replica}.xtc")

n_frames = len(u.trajectory)
dt = u.trajectory.dt
time = np.arange(n_frames) * dt

rmsd_values = np.zeros(n_frames - 1)

ref = u.copy()
ref.trajectory[0]

aligner = align.AlignTraj(u, ref, select='backbone', in_memory=True)
aligner.run()

prev_pos = None
for i, ts in enumerate(u.trajectory):
    if i > 0:
        current_pos = u.select_atoms('backbone').positions
        rmsd = rms.rmsd(current_pos, prev_pos)
        rmsd_values[i-1] = rmsd
    prev_pos = u.select_atoms('backbone').positions.copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time[1:], y=rmsd_values,
                        mode='lines',
                        name='RMSD'))

fig.update_layout(
    title=f'{name} RMSD Between Consecutive Frames',
    xaxis_title='Time (ps)',
    yaxis_title='RMSD (Å)',
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [ ]:
# Calculate hamming distances between consecutive frames
distances = []
dist_name = "Levenshtein"
for i in range(1, len(tokenized_mdcath[f'{name}_{temp}_{replica}'])):
    distance = levenshtein_distance(tokenized_mdcath[f'{name}_{temp}_{replica}'][i-1], tokenized_mdcath[f'{name}_{temp}_{replica}'][i])
    distances.append(distance)

fig = go.Figure()
fig.add_trace(go.Scatter(x=time[1:], y=distances,
                        mode='lines',
                        name='Distance'))

fig.update_layout(
    title=f'{dist_name} Distance Between Consecutive Frames',
    xaxis_title='Time ', 
    yaxis_title=f'{dist_name} Distance',
    showlegend=True,
    template='plotly_white'
)

fig.show()


In [ ]:
fig = go.Figure()

start_idx = 1
fig.add_trace(go.Scatter(x=time[start_idx:], y=rmsd_values[start_idx:],
                        mode='lines',
                        name='RMSD',
                        yaxis='y2'))
fig.add_trace(go.Scatter(x=time[start_idx:], y=distances[start_idx:],
                        mode='lines',
                        name=dist_name))

fig.update_layout(
    yaxis2=dict(
        overlaying='y',
        side='right',
        title='RMSD (Å)'
    )
)

fig.update_layout(
    yaxis=dict(
        title=f'{dist_name} Distance'
    )
)

fig.update_layout(
    title='MD RMSD and Token Levenshtein Distance vs Previous Frame',
    xaxis_title='Time (ps)',
    showlegend=True,
    template='plotly_white'
)

# Sequence Motifs

In [ ]:
tokenized_mdcath['12asA00_320_0']

In [ ]:
import json
import pandas as pd
import logomaker as lm
import matplotlib.pyplot as plt

# trajectory_name = "1a7mA00_320_4"
for trajectory_name in list(tokenized_mdcath.keys())[10:]:
    sequences = tokenized_mdcath[trajectory_name]

    # Extract the list of sequences
    sequence_list = sequences

    # Count the frequency of each amino acid at each position
    position_counts = pd.DataFrame()

    # Get the maximum sequence length
    max_length = max(len(seq) for seq in sequence_list)

    # Initialize the count dataframe
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    position_counts = pd.DataFrame(0, index=list(amino_acids), columns=range(max_length))

    # Count the amino acid occurrences at each position
    for seq in sequence_list:
        for i, aa in enumerate(seq):
            if aa in position_counts.index:
                position_counts.loc[aa, i] += 1

    # Normalize the counts to frequencies
    position_frequencies = position_counts.div(position_counts.sum(axis=0), axis=1).fillna(0)

    # Convert frequencies to log odds (PSSM style)
    background_freq = 1 / len(amino_acids)  # Assume uniform background distribution
    position_log_odds = position_frequencies.map(lambda x: 0 if x == 0 else np.log2(x / background_freq))

    # Transpose to match Logomaker's expected format
    position_log_odds_t = position_log_odds.T

    # Create a high resolution figure
    plt.figure(figsize=(16, 9), dpi=1200)  # Increased DPI for higher resolution

    # Create a sequence logo with higher resolution
    logo = lm.Logo(position_log_odds_t, vsep=0.01)  # Reduced vertical separation for cleaner look

    # Customize the appearance of the logo
    logo.style_spines(visible=False)
    logo.style_spines(spines=['left', 'bottom'], visible=True)

    # Only show x-ticks every 10 positions
    xticks = list(range(0, max_length, 10))
    logo.ax.set_xticks(xticks)
    logo.ax.set_xticklabels([str(x) for x in xticks], rotation=90)

    # Increase font sizes for better readability in high resolution
    plt.title(f'Sequence Logo (PSSM) for {trajectory_name}', pad=20, fontsize=14)
    plt.xlabel('Position', fontsize=12)
    plt.ylabel('Log-Odds Score', fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    # Ensure the figure is rendered at high quality
    plt.rcParams['figure.dpi'] = 1200
    plt.rcParams['savefig.dpi'] = 1200

    plt.savefig(f'../tmp/plots/sequence_logo_{trajectory_name}.png', dpi=600, bbox_inches='tight')

    plt.tight_layout()
    plt.show()
